In [1]:
from dotenv import load_dotenv  # type: ignore
import jupyter_black
load_dotenv()
jupyter_black.load()

In [2]:
import os
from baml_agents import with_hooks
from baml_client.sync_client import b as sb
from baml_client.async_client import b as ab
from openai import AsyncOpenAI

client = AsyncOpenAI(base_url=os.getenv("OPENAI_API_BASE"))

In [3]:
input_text = "i'm john from chicago"
expected_name = "Chicago"

b = with_hooks(sb, [])
actual = b.TestBamlHooks_ExtractCity(input_text)
assert actual.name == expected_name, f"Expected {expected_name}, got {actual.name}"

b = with_hooks(ab, [])
actual = await b.TestBamlHooks_ExtractCity(input_text)
assert actual.name == expected_name, f"Expected {expected_name}, got {actual.name}"

b = with_hooks(sb, [])
request = b.request.TestBamlHooks_ExtractCity(input_text)
response = await client.chat.completions.create(**request.body.json())
actual = b.parse.TestBamlHooks_ExtractCity(response.choices[0].message.content)
assert actual.name == expected_name, f"Expected {expected_name}, got {actual.name}"

b = with_hooks(ab, [])
request = await b.request.TestBamlHooks_ExtractCity(input_text)
response = await client.chat.completions.create(**request.body.json())
actual = b.parse.TestBamlHooks_ExtractCity(response.choices[0].message.content)
assert actual.name == expected_name, f"Expected {expected_name}, got {actual.name}"

In [4]:
import pytest

b = with_hooks(sb, [])
with pytest.raises(AttributeError):
    b = b.with_options()

In [8]:
from baml_py import ClientRegistry
from baml_agents import with_hooks, WithOptions
from baml_agents._baml_client_proxy._hooks._base_hook import BaseBamlHookSync
from baml_client.sync_client import b

cr = ClientRegistry()
cr.add_llm_client("MyClient", "openai", options={"model": "first"})
cr.set_primary("MyClient")


class WithOptions1(WithOptions):
    pass


# Replaces b.with_options(client_registry=cr)
b = with_hooks(
    b, [BaseBamlHookSync(), WithOptions1(client_registry=cr), BaseBamlHookSync()]
)

cr = ClientRegistry()
cr.add_llm_client("MyClient", "openai", options={"model": "second"})
cr.set_primary("MyClient")


class WithOptions2(WithOptions):
    pass


# Replaces b.with_options(client_registry=cr)
b = with_hooks(
    b, [BaseBamlHookSync(), WithOptions2(client_registry=cr), BaseBamlHookSync()]
)

request = b.request.TestBamlHooks_ExtractEntities("my name is john and im from NY")
assert request.body.json()["model"] == "second"

WithOptions1
WithOptions2
BaseBamlHookSync
BaseBamlHookSync
BaseBamlHookSync
BaseBamlHookSync
